In [ ]:
!pip install -q tensorflow==2.15.0 tensorflow-addons==0.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=

In [ ]:
!pip install -q -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ------------------------
# 1. Imports and Setup
# ------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize, pos_tag
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Concatenate
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow_addons.layers import CRF
from tensorflow_addons.text import crf_decode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing Data

train = pd.read_csv('/content/drive/MyDrive/SJ-DECEN_code/data/train_cleaned_no_punkt.csv')
test_labelled = pd.read_csv('/content/drive/MyDrive/SJ-DECEN_code/data/test_labelled_cleaned_no_punkt.csv')
test_unlabelled = pd.read_csv('/content/drive/MyDrive/SJ-DECEN_code/data/test_unlabelled_cleaned_no_punkt.csv')

In [ ]:
print("TF version:", tf.__version__)
print("TFA version:", tfa.__version__)

TF version: 2.15.0
TFA version: 0.22.0


In [ ]:
# Download NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# ------------------------
# 2. Load & Prepare Data
# ------------------------

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

for df in [train, test_labelled]:
    df['mal'] = (df[label_cols].sum(axis=1) >= 1).astype(int)
    df.drop(columns=label_cols, inplace=True)
    df['comment_text'].fillna("empty", inplace=True)

test_unlabelled['comment_text'].fillna("empty", inplace=True)

# Combine train and test_labelled for stratified splitting
combined_df = pd.concat([train, test_labelled], axis=0).reset_index(drop=True)

X_temp, X_test, y_temp, y_test = train_test_split(
    combined_df['comment_text'], combined_df['mal'],
    test_size=0.2, stratify=combined_df['mal'], random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.1, stratify=y_temp, random_state=42)


## SJ-DECEN Module 1

In [ ]:
# ---------------------------
# 3. Load & Filter NRC EmoLex
# ---------------------------

nrc_emolex = pd.read_csv("/content/drive/MyDrive/SJ-DECEN_code/Model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt", sep='\t', names=['word', 'emotion', 'association'])
nrc_emolex = nrc_emolex[nrc_emolex['association'] == 1]
emotion_dict = nrc_emolex.groupby('word')['emotion'].apply(list).to_dict()

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
# ---------------------------
# 4. Token Labeling Function
# ---------------------------

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

def label_tokens_with_emotions(text, emotion_dict):
    tokens = word_tokenize(text.lower())
    pos_tags = pos_tag(tokens)
    labels = [ '|'.join(emotion_dict[token]) if token in emotion_dict else 'O' for token in tokens ]
    return tokens, pos_tags, labels

combined_df['tokens_pos_labels'] = combined_df['comment_text'].apply(lambda x: label_tokens_with_emotions(x, emotion_dict))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
combined_df.head()

,id,comment_text,mal,tokens_pos_labels
0,0000997932d777bf,explanation why edits made under my username h...,0,"([explanation, why, edits, made, under, my, us..."
1,000103f0d9cfb60f,d aww he matches this background colour i seem...,0,"([d, aww, he, matches, this, background, colou..."
2,000113f07ec002fd,hey man i really not trying to edit war just t...,0,"([hey, man, i, really, not, trying, to, edit, ..."
3,0001b41b1c6bb37e,more i can not make any real suggestions impro...,0,"([more, i, can, not, make, any, real, suggesti..."
4,0001d958c54c6e35,you sir my hero any chance you remember what p...,0,"([you, sir, my, hero, any, chance, you, rememb..."


In [ ]:
# --------------------------
# 5. Tokenizer and POS Vocabulary
# --------------------------

all_tokens = set(token for tokens, _, _ in combined_df['tokens_pos_labels'] for token in tokens)
word2idx = {word: i+1 for i, word in enumerate(sorted(all_tokens))}
word2idx['PAD'] = 0

def tokens_to_seq(tokens):
    return [word2idx.get(token, 0) for token in tokens]

# POS vocab
all_pos = set()
for _, pos_tags, _ in combined_df['tokens_pos_labels']:
    all_pos.update([pos for _, pos in pos_tags])
pos2idx = {pos: i+1 for i, pos in enumerate(sorted(all_pos))}
pos2idx['PAD'] = 0

def pos_tags_to_seq(pos_tags):
    return [pos2idx.get(pos, 0) for _, pos in pos_tags]

# Label encoding
all_labels = set()
for _, _, labels in combined_df['tokens_pos_labels']:
    all_labels.update(labels)
label2idx = {label: i for i, label in enumerate(sorted(all_labels))}
idx2label = {i: label for label, i in label2idx.items()}

def labels_to_seq(labels):
    return [label2idx[label] for label in labels]

In [ ]:
print(f"Number of words: {len(word2idx)}")
print(f"Number of POS tags: {len(pos2idx)}")
print(f"Number of labels: {len(label2idx)}")

Number of words: 224405
Number of POS tags: 40
Number of labels: 240


In [ ]:
# --------------------------
# 6. Prepare Sequences for DER
# --------------------------
max_len = 100

def prepare_inputs(df):
    word_seq = [tokens_to_seq(tokens) for tokens, _, _ in df['tokens_pos_labels']]
    pos_seq = [pos_tags_to_seq(pos_tags) for _, pos_tags, _ in df['tokens_pos_labels']]
    label_seq = [labels_to_seq(labels) for _, _, labels in df['tokens_pos_labels']]

    X_w = pad_sequences(word_seq, maxlen=max_len, padding='post')
    X_p = pad_sequences(pos_seq, maxlen=max_len, padding='post')
    y = pad_sequences(label_seq, maxlen=max_len, padding='post', value=label2idx['O'])
    return X_w, X_p, y


In [ ]:
# Sample only 5% of training data
five_percent_idx = X_train.sample(frac=0.05, random_state=42).index
X_word_train, X_pos_train, y_train_der = prepare_inputs(combined_df.loc[five_percent_idx])

X_word_val, X_pos_val, y_val_der = prepare_inputs(combined_df.loc[X_val.index])
X_word_test, X_pos_test, y_test_der = prepare_inputs(combined_df.loc[X_test.index])

In [ ]:
sampled_df = combined_df.loc[five_percent_idx]
print(sampled_df['mal'].value_counts(normalize=True))

mal
0    0.901093
1    0.098907
Name: proportion, dtype: float64


In [ ]:
# --------------------------
# 7. Build & Train DER BiLSTM-CRF
# --------------------------

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Concatenate, Layer

# Custom CRF layer
class CRFLayer(Layer):
    def __init__(self, num_labels):
        super(CRFLayer, self).__init__()
        self.num_labels = num_labels

    def build(self, input_shape):
        self.transitions = self.add_weight(
            name="transitions",
            shape=(self.num_labels, self.num_labels),
            initializer="glorot_uniform",
            trainable=True
        )

    def call(self, logits, mask=None):
        sequence_lengths = tf.math.reduce_sum(tf.cast(mask, tf.int32), axis=1)
        decoded_tags, _ = tfa.text.crf_decode(logits, self.transitions, sequence_lengths)
        return decoded_tags

    def compute_loss(self, y_true, y_pred, mask):
        log_likelihood, _ = tfa.text.crf_log_likelihood(
            inputs=y_pred,
            tag_indices=y_true,
            sequence_lengths=tf.math.reduce_sum(tf.cast(mask, tf.int32), axis=1),
            transition_params=self.transitions
        )
        return -tf.reduce_mean(log_likelihood)

# Model hyperparameters
vocab_size = len(word2idx) + 1
pos_vocab_size = len(pos2idx)
embedding_dim = 100
pos_embedding_dim = 25
num_labels = len(label2idx)
max_len = X_word_train.shape[1]

# Inputs
word_in = Input(shape=(max_len,), name='word_input')
pos_in = Input(shape=(max_len,), name='pos_input')

# Mask for padding (for CRF)
mask = tf.cast(tf.math.not_equal(word_in, 0), tf.float32)

# Embeddings
word_emb = Embedding(vocab_size, embedding_dim, mask_zero=True)(word_in)
pos_emb = Embedding(pos_vocab_size, pos_embedding_dim, mask_zero=True)(pos_in)
x = Concatenate()([word_emb, pos_emb])
x = Bidirectional(LSTM(64, return_sequences=True))(x)
logits = Dense(num_labels)(x)

# Instantiate CRF layer
crf_layer = CRFLayer(num_labels)
decoded_tags = crf_layer(logits, mask=mask)

# Build model for prediction
der_model = Model(inputs=[word_in, pos_in], outputs=decoded_tags)

# Custom loss function for training
def custom_crf_loss(y_true, y_pred_logits):
    # `y_pred_logits` is the pre-softmax output from the Dense layer (logits)
    mask_ = tf.cast(tf.math.not_equal(y_true, 0), tf.float32)
    return crf_layer.compute_loss(y_true, y_pred_logits, mask_)

# Wrap logits in a separate model for training
training_model = Model(inputs=[word_in, pos_in], outputs=logits)
training_model.compile(optimizer='adam', loss=custom_crf_loss)

with tf.device('/GPU:0'):
    training_model.fit(
        [X_word_train, X_pos_train], y_train_der,
        batch_size=32, epochs=5,
        validation_data=([X_word_val, X_pos_val], y_val_der)
    )


Epoch 1/5
252/252 [==============================] - 784s 3s/step - loss: 5.7539 - val_loss: 3.2925
Epoch 2/5
252/252 [==============================] - 718s 3s/step - loss: 2.5427 - val_loss: 2.3948
Epoch 3/5
252/252 [==============================] - 730s 3s/step - loss: 1.8990 - val_loss: 2.0399
Epoch 4/5
252/252 [==============================] - 750s 3s/step - loss: 1.4146 - val_loss: 1.7103
Epoch 5/5
252/252 [==============================] - 794s 3s/step - loss: 1.0038 - val_loss: 1.5520


In [ ]:
def predict_depressive_emotions(text):
    tokens, pos_tags, _ = label_tokens_with_emotions(text, emotion_dict)

    word_seq = [tokens_to_seq(tokens)]
    pos_seq = [pos_tags_to_seq(pos_tags)]

    word_seq_padded = pad_sequences(word_seq, maxlen=max_len, padding='post')
    pos_seq_padded = pad_sequences(pos_seq, maxlen=max_len, padding='post')

    # Already decoded output
    pred_labels = der_model.predict([word_seq_padded, pos_seq_padded], verbose=0)[0][:len(tokens)]

    # Convert predicted indices to emotion labels
    labeled_tokens = [(token, idx2label[int(label)]) for token, label in zip(tokens, pred_labels)]
    return labeled_tokens


In [ ]:
# Predict on validation/test data
y_pred_der = der_model.predict([X_word_val, X_pos_val])

559/559 [==============================] - 276s 490ms/step


In [ ]:
def decode_predictions(preds, ground_truths):
    decoded_preds, decoded_true = [], []

    for pred_seq, true_seq in zip(preds, ground_truths):
        decoded_seq, true_seq_decoded = [], []
        for p, t in zip(pred_seq, true_seq):
            if t != label2idx['O']:  # Skip padding or neutral label if needed
                decoded_seq.append(idx2label[p])
                true_seq_decoded.append(idx2label[t])
        decoded_preds.append(decoded_seq)
        decoded_true.append(true_seq_decoded)

    return decoded_preds, decoded_true

decoded_preds, decoded_true = decode_predictions(y_pred_der, y_val_der)


In [ ]:
# Save the decoded tag output model
der_model.save("/content/drive/MyDrive/SJ-DECEN_code/Model/der_model.h5")

# Save the logits output model used for emotion contextual representation
emotion_model = training_model  # Save logits model for ECR
emotion_model.save("/content/drive/MyDrive/SJ-DECEN_code/Model/emotion_logits_model.h5")

In [ ]:
# Saving results

# Save mappings to Drive
import json

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/word2idx.json', 'w') as f:
    json.dump(word2idx, f)

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/pos2idx.json', 'w') as f:
    json.dump(pos2idx, f)

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/label2idx.json', 'w') as f:
    json.dump(label2idx, f)

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/emotion_dict.json', 'w') as f:
    json.dump(emotion_dict, f)


In [ ]:
import json

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/word2idx.json') as f:
    word2idx = json.load(f)

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/pos2idx.json') as f:
    pos2idx = json.load(f)

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/label2idx.json') as f:
    label2idx = json.load(f)

with open('/content/drive/MyDrive/SJ-DECEN_code/Model/emotion_dict.json') as f:
    emotion_dict = json.load(f)

idx2label = {v: k for k, v in label2idx.items()}


## SJ-DECEN Module 2



In [ ]:
# ---------------------------
# 1. Importing relevant packages and models
# ---------------------------

from tensorflow.keras.models import load_model
emotion_model = load_model('/content/drive/MyDrive/SJ-DECEN_code/Model/emotion_logits_model.h5', compile=False)


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import os

In [ ]:
# ------------------------------------------------
# 2. Define Batch-wise BERT Representation and Emotion Representation
# ------------------------------------------------

def batchwise_bert_encode(sentences, model, batch_size=256, cache_path=None):
    all_embeddings = []

    for i in tqdm(range(0, len(sentences), batch_size), desc="BERT Encoding"):
        batch = sentences[i:i + batch_size]
        embeddings = model.encode(batch, show_progress_bar=False)
        all_embeddings.append(embeddings)

        # Optional: Cache after every batch
        if cache_path:
            partial_path = os.path.join(cache_path, f'batch_{i//batch_size}.npy')
            np.save(partial_path, embeddings)

    return np.vstack(all_embeddings)


In [ ]:
from tqdm import tqdm
import numpy as np
import os

def batchwise_emotion_encode(X_word, X_pos, model, batch_size=256, cache_path=None):

    all_emotion_vecs = []
    num_samples = len(X_word)

    if cache_path:
        os.makedirs(cache_path, exist_ok=True)

    for i in tqdm(range(0, num_samples, batch_size), desc="Emotion Encoding"):
        batch_word = X_word[i:i + batch_size]
        batch_pos = X_pos[i:i + batch_size]

        # Predict token-level emotion outputs
        batch_emotion_preds = model.predict([batch_word, batch_pos], batch_size=batch_size, verbose=0)

        # Mean-pooling over tokens → sequence-level emotion vector
        batch_emotion_vecs = batch_emotion_preds.mean(axis=1)

        all_emotion_vecs.append(batch_emotion_vecs)

        # Optional caching
        # if cache_path:
        #     np.save(os.path.join(cache_path, f'batch_{i//batch_size}.npy'), batch_emotion_vecs)

    return np.vstack(all_emotion_vecs)


In [ ]:
# -------------
# 3. Load BERT 
# -------------

bert_model = SentenceTransformer('all-MiniLM-L6-v2')  # much faster and lighter

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ---------------------------
# 4. Generating bert represenations and emotion labels
# ---------------------------

# Input: full-text posts (strings)
X_train_text = X_train.tolist()
X_val_text = X_val.tolist()
X_test_text = X_test.tolist()

# For Module 2 usage (emotion representation for full training set)
X_word_train_full, X_pos_train_full, _ = prepare_inputs(combined_df.loc[X_train.index])
X_word_val_full, X_pos_val_full, _ = prepare_inputs(combined_df.loc[X_val.index])
X_word_test_full, X_pos_test_full, _ = prepare_inputs(combined_df.loc[X_test.index])

In [ ]:
emotion_vecs_train = batchwise_emotion_encode(
    X_word_train_full, X_pos_train_full, emotion_model,
    batch_size=256
    # cache_path='/content/drive/MyDrive/SJ-DECEN_code/Model/EmotionCache/Train'
)

emotion_vecs_val = batchwise_emotion_encode(
    X_word_val_full, X_pos_val_full, emotion_model,
    batch_size=256
    # cache_path='/content/drive/MyDrive/SJ-DECEN_code/Model/EmotionCache/Val'
)

emotion_vecs_test = batchwise_emotion_encode(
    X_word_test_full, X_pos_test_full, emotion_model,
    batch_size=256
    # cache_path='/content/drive/MyDrive/SJ-DECEN_code/Model/EmotionCache/Test'
)


Emotion Encoding: 100%|██████████| 175/175 [01:01<00:00,  2.85it/s]


In [ ]:
# Run BERT
bert_vecs_train = batchwise_bert_encode(X_train_text, bert_model)
bert_vecs_val = batchwise_bert_encode(X_val_text, bert_model)
bert_vecs_test = batchwise_bert_encode(X_test_text, bert_model)

BERT Encoding: 100%|██████████| 175/175 [00:39<00:00,  4.44it/s]


In [ ]:
# ---------------------------
# 5. Concatenating the two inputs
# ---------------------------

# Align length if needed
emotion_vecs_train = emotion_vecs_train[:len(bert_vecs_train)]
emotion_vecs_val = emotion_vecs_val[:len(bert_vecs_val)]
emotion_vecs_test = emotion_vecs_test[:len(bert_vecs_test)]

# Concatenate
ECER_train = np.concatenate([bert_vecs_train, emotion_vecs_train], axis=1)
ECER_val = np.concatenate([bert_vecs_val, emotion_vecs_val], axis=1)
ECER_test = np.concatenate([bert_vecs_test, emotion_vecs_test], axis=1)

## SJ-DECEN MODULE 3

In [ ]:
# ---------------------------
# 1. BiLSTM layer + FFNN
# ---------------------------

from tensorflow.keras import layers, models

# Define input
input_dim = ECER_train.shape[1]
input_layer = Input(shape=(input_dim,))

# Expand dims for LSTM
reshape = layers.Reshape((1, input_dim))(input_layer)

# BiLSTM Layer
lstm = Bidirectional(LSTM(64, return_sequences=False))(reshape)
drop = Dropout(0.3)(lstm)

# Classifier
output = Dense(1, activation='sigmoid')(drop)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 624)]             0         
                                                                 
 reshape (Reshape)           (None, 1, 624)            0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               352768    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 352897 (1.35 MB)
Trainable params: 352897 (1.35 MB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(ECER_train, y_train.values,
          epochs=20,
          batch_size=16,
          validation_data=(ECER_val, y_val.values),
          callbacks=[early_stop])

Epoch 1/20
10060/10060 [==============================] - 55s 5ms/step - loss: 0.2032 - accuracy: 0.9256 - val_loss: 0.1621 - val_accuracy: 0.9354
Epoch 2/20
10060/10060 [==============================] - 52s 5ms/step - loss: 0.1769 - accuracy: 0.9326 - val_loss: 0.1670 - val_accuracy: 0.9342
Epoch 3/20
10060/10060 [==============================] - 53s 5ms/step - loss: 0.1713 - accuracy: 0.9353 - val_loss: 0.1657 - val_accuracy: 0.9346
Epoch 4/20
10060/10060 [==============================] - 58s 6ms/step - loss: 0.1688 - accuracy: 0.9361 - val_loss: 0.1727 - val_accuracy: 0.9321


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, average_precision_score

# Predictions
y_pred_test = (model.predict(ECER_test) > 0.5).astype(int)

# Evaluation
print("Test Accuracy:", accuracy_score(y_test.values, y_pred_test))
print(classification_report(y_test.values, y_pred_test))
print("Average Precision Score:", average_precision_score(y_test.values, y_pred_test))


1398/1398 [==============================] - 3s 2ms/step
Test Accuracy: 0.9381570118541713
              precision    recall  f1-score   support

           0       0.95      0.98      0.97     40216
           1       0.75      0.58      0.65      4494

    accuracy                           0.94     44710
   macro avg       0.85      0.78      0.81     44710
weighted avg       0.93      0.94      0.93     44710

Average Precision Score: 0.4756707872423106


### Verifying/Improving the above 75% precision


In [ ]:
# Ensuring that the predictions are not 0 0 ... which indicates that the model has not been built properly.

In [ ]:
import numpy as np
print("Train label distribution:", np.bincount(y_train.values))
print("Test label distribution:", np.bincount(y_test.values))

y_pred = (model.predict(ECER_test) > 0.5).astype(int)
print("Predicted labels:", np.bincount(y_pred.flatten()))


Train label distribution: [144778  16177]
Test label distribution: [40216  4494]
1398/1398 [==============================] - 3s 2ms/step
Predicted labels: [41241  3469]


In [ ]:
print("BERT vector train mean/std:", np.mean(bert_vecs_train), np.std(bert_vecs_train))
print("Emotion vector train mean/std:", np.mean(emotion_vecs_train), np.std(emotion_vecs_train))


BERT vector train mean/std: -0.0005463283 0.051028095
Emotion vector train mean/std: -2.151786 2.3514543


In [ ]:
# Standardising emotion vector and applying CNN instead of FFNN

# # From the previous result:
# Emotion vector mean and std: Drastically different scale — this is a problem.

# Emotion vectors are overpowering the BERT vectors in the concatenated ECER input.

### Standardising the Emotion vectors and running Module 3



In [ ]:
# -------------------------------------------------
# 2. Applying BiLSTM layer + FFNN to Standardised Emotion Vectors
# --------------------------------------------------

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
emotion_vecs_train = scaler.fit_transform(emotion_vecs_train)
emotion_vecs_val = scaler.transform(emotion_vecs_val)
emotion_vecs_test = scaler.transform(emotion_vecs_test)

In [ ]:
ECER_train = np.concatenate([bert_vecs_train, emotion_vecs_train], axis=1)
ECER_val = np.concatenate([bert_vecs_val, emotion_vecs_val], axis=1)
ECER_test = np.concatenate([bert_vecs_test, emotion_vecs_test], axis=1)


In [ ]:
## Modifying the 75% one
from sklearn.preprocessing import StandardScaler
import numpy as np

# Fit only on training emotion vectors
scaler = StandardScaler()
emotion_vecs_train_scaled = scaler.fit_transform(emotion_vecs_train)
emotion_vecs_val_scaled = scaler.transform(emotion_vecs_val)
emotion_vecs_test_scaled = scaler.transform(emotion_vecs_test)

# Concatenate with BERT
ECER_train = np.concatenate([bert_vecs_train, emotion_vecs_train_scaled], axis=1)
ECER_val = np.concatenate([bert_vecs_val, emotion_vecs_val_scaled], axis=1)
ECER_test = np.concatenate([bert_vecs_test, emotion_vecs_test_scaled], axis=1)


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Reshape, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Define input
input_dim = ECER_train.shape[1]
input_layer = Input(shape=(input_dim,))

# Reshape for LSTM
reshape = layers.Reshape((1, input_dim))(input_layer)

# BiLSTM Layer
lstm = Bidirectional(LSTM(64, return_sequences=False))(reshape)
drop = Dropout(0.3)(lstm)

# Classifier
output = Dense(1, activation='sigmoid')(drop)

# Build model
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
model.fit(ECER_train, y_train.values,
          epochs=20,
          batch_size=16,
          validation_data=(ECER_val, y_val.values),
          callbacks=[early_stop])


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 624)]             0         
                                                                 
 reshape_1 (Reshape)         (None, 1, 624)            0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 128)               352768    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 352897 (1.35 MB)
Trainable params: 352897 (1.35 MB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [ ]:
# Evaluate
y_pred_prob = model.predict(ECER_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("Test Accuracy:", accuracy_score(y_test.values, y_pred))
print("Classification Report:\n", classification_report(y_test.values, y_pred))
print("Average Precision Score:", average_precision_score(y_test.values, y_pred_prob))

1398/1398 [==============================] - 3s 2ms/step
Test Accuracy: 0.9451353164840081
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97     40216
           1       0.78      0.63      0.70      4494

    accuracy                           0.95     44710
   macro avg       0.87      0.81      0.83     44710
weighted avg       0.94      0.95      0.94     44710

Average Precision Score: 0.7940326707784715


## Improvising concatenation of inputs for module 3

In [ ]:
# ------------------------------------------------
# 3. Residual dot product to concatenate two inputs for BiLSTM layer + FFNN
# -------------------------------------------------

## Modifying the 75% one
from sklearn.preprocessing import StandardScaler
import numpy as np

# Fit only on training emotion vectors
scaler = StandardScaler()
emotion_vecs_train_scaled = scaler.fit_transform(emotion_vecs_train)
emotion_vecs_val_scaled = scaler.transform(emotion_vecs_val)
emotion_vecs_test_scaled = scaler.transform(emotion_vecs_test)

from sklearn.decomposition import PCA

# Step 1: Reduce BERT to 240D to match emotion dimension
pca_bert = PCA(n_components=240)
bert_vecs_train_proj = pca_bert.fit_transform(bert_vecs_train)
bert_vecs_val_proj   = pca_bert.transform(bert_vecs_val)
bert_vecs_test_proj  = pca_bert.transform(bert_vecs_test)



def residual_dot_fusion(bert_vecs_proj, emotion_vecs):
    dot_scores = np.einsum('ij,ij->i', bert_vecs_proj, emotion_vecs).reshape(-1, 1)
    fused = bert_vecs_proj + (bert_vecs_proj * dot_scores)
    return fused

ECER_train = residual_dot_fusion(bert_vecs_train_proj, emotion_vecs_train_scaled)
ECER_val   = residual_dot_fusion(bert_vecs_val_proj, emotion_vecs_val_scaled)
ECER_test  = residual_dot_fusion(bert_vecs_test_proj, emotion_vecs_test_scaled)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

input_dim = ECER_train.shape[1]

# Input layer
input_layer = Input(shape=(input_dim,))

# Reshape to (timesteps=1, features)
reshaped = Reshape((1, input_dim))(input_layer)

# BiLSTM
lstm_out = Bidirectional(LSTM(64, return_sequences=False))(reshaped)

# Feedforward classifier
fc1 = Dense(64, activation='relu')(lstm_out)
drop = Dropout(0.3)(fc1)
fc2 = Dense(32, activation='relu')(drop)
output = Dense(1, activation='sigmoid')(fc2)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 240)]             0         
                                                                 
 reshape_2 (Reshape)         (None, 1, 240)            0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               156160    
 onal)                                                           
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                           

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    ECER_train, y_train.values,
    validation_data=(ECER_val, y_val.values),
    epochs=20, batch_size=16,
    callbacks=[early_stop]
)


Epoch 1/20
10060/10060 [==============================] - 53s 4ms/step - loss: 0.1916 - accuracy: 0.9325 - val_loss: 0.1736 - val_accuracy: 0.9350
Epoch 2/20
10060/10060 [==============================] - 37s 4ms/step - loss: 0.1675 - accuracy: 0.9400 - val_loss: 0.1654 - val_accuracy: 0.9392
Epoch 3/20
10060/10060 [==============================] - 39s 4ms/step - loss: 0.1573 - accuracy: 0.9429 - val_loss: 0.1657 - val_accuracy: 0.9378
Epoch 4/20
10060/10060 [==============================] - 37s 4ms/step - loss: 0.1489 - accuracy: 0.9459 - val_loss: 0.1606 - val_accuracy: 0.9398
Epoch 5/20
10060/10060 [==============================] - 36s 4ms/step - loss: 0.1403 - accuracy: 0.9487 - val_loss: 0.1593 - val_accuracy: 0.9420
Epoch 6/20
10060/10060 [==============================] - 38s 4ms/step - loss: 0.1316 - accuracy: 0.9513 - val_loss: 0.1618 - val_accuracy: 0.9418
Epoch 7/20
10060/10060 [==============================] - 38s 4ms/step - loss: 0.1237 - accuracy: 0.9550 - val_loss: 0

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Predict on the test set
y_pred_prob = model.predict(ECER_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()  # Convert probabilities to binary labels

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


1398/1398 [==============================] - 3s 2ms/step
Accuracy: 0.943099977633639

Confusion Matrix:
 [[39626   590]
 [ 1954  2540]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97     40216
           1       0.81      0.57      0.67      4494

    accuracy                           0.94     44710
   macro avg       0.88      0.78      0.82     44710
weighted avg       0.94      0.94      0.94     44710



## Hybrid CNN + BiLSTM 

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Reshape, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

input_dim = ECER_train.shape[1]
input_layer = Input(shape=(input_dim,), name='input')

# Project to lower-dimensional space
projected = Dense(128, activation='relu', name='projection')(input_layer)

# Reshape into smaller pseudo-sequence: (16 steps, 8 dims)
reshaped = Reshape((16, 8), name='reshaped')(projected)

# Multi-scale CNNs
conv3 = Conv1D(64, kernel_size=3, activation='relu', padding='same')(reshaped)
conv4 = Conv1D(64, kernel_size=4, activation='relu', padding='same')(reshaped)
conv5 = Conv1D(64, kernel_size=5, activation='relu', padding='same')(reshaped)

conv_concat = Concatenate(name='conv_concat')([conv3, conv4, conv5])
conv_concat = BatchNormalization()(conv_concat)

# BiLSTM over small sequence
bilstm = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2), name='bilstm')(conv_concat)

# Pooling
max_pool = GlobalMaxPooling1D()(bilstm)
avg_pool = GlobalAveragePooling1D()(bilstm)
combined_pool = Concatenate(name='pool_concat')([max_pool, avg_pool])

# Classification head
x = Dense(128, activation='relu', kernel_initializer='he_normal')(combined_pool)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
x = Dropout(0.2)(x)

# Output
output = Dense(1, activation='sigmoid', name='output')(x)

# Compile
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fit (this will run much faster)
model.fit(ECER_train.astype('float32'), y_train.values,
          epochs=20,
          batch_size=16,
          validation_data=(ECER_val.astype('float32'), y_val.values),
          callbacks=[early_stop])


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 240)]                0         []                            
                                                                                                  
 projection (Dense)          (None, 128)                  30848     ['input[0][0]']               
                                                                                                  
 reshaped (Reshape)          (None, 16, 8)                0         ['projection[0][0]']          
                                                                                                  
 conv1d_1 (Conv1D)           (None, 16, 64)               1600      ['reshaped[0][0]']            
                                                                                           

In [ ]:
from sklearn.metrics import classification_report, average_precision_score, accuracy_score

# Evaluate
y_pred_prob = model.predict(ECER_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("Test Accuracy:", accuracy_score(y_test.values, y_pred))
print("Classification Report:\n", classification_report(y_test.values, y_pred))
print("Average Precision Score:", average_precision_score(y_test.values, y_pred_prob))

1398/1398 [==============================] - 10s 7ms/step
Test Accuracy: 0.9433236412435697
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97     40216
           1       0.83      0.55      0.66      4494

    accuracy                           0.94     44710
   macro avg       0.89      0.77      0.82     44710
weighted avg       0.94      0.94      0.94     44710

Average Precision Score: 0.7712759677148184
